In [1]:
import xarray as xr
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier

from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from imblearn import pipeline

from imblearn.under_sampling import ClusterCentroids, RandomUnderSampler, TomekLinks, NearMiss

from tqdm.notebook import tqdm

sns.set_theme('notebook')

In [2]:
data = xr.open_dataset('data/oaflux_air_sea_fluxes_train.nc', engine='netcdf4')
labels = xr.open_dataset('data/marine_heatwave_labels_train.nc', engine='netcdf4')

In [3]:
df = None
variables = list(data.variables)
for var in tqdm(variables[3:]): 
    
    stacked = data[var].stack(dim=["lon", "lat", "time"]).to_pandas().T
    if df is None:
        df = stacked
    else:
        df = pd.concat([df, stacked], axis=1)

df.reset_index(inplace=True)
df.columns = variables

  0%|          | 0/10 [00:00<?, ?it/s]

In [4]:
# sort variables and remove rows where ALL variables are NaN
df = df.sort_values(by=["time"], ascending=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26697600 entries, 0 to 26697599
Data columns (total 13 columns):
 #   Column  Dtype         
---  ------  -----         
 0   lon     float64       
 1   lat     float64       
 2   time    datetime64[ns]
 3   evapr   float32       
 4   lhtfl   float32       
 5   nlwrs   float32       
 6   hum2m   float32       
 7   qnet    float32       
 8   shtfl   float32       
 9   nswrs   float32       
 10  tmp2m   float32       
 11  tmpsf   float32       
 12  wnd10   float32       
dtypes: datetime64[ns](1), float32(10), float64(2)
memory usage: 1.8 GB


In [5]:
df["year"] = df.time.dt.year
df["month"] = df.time.dt.month
df = df.drop(["time"], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26697600 entries, 0 to 26697599
Data columns (total 14 columns):
 #   Column  Dtype  
---  ------  -----  
 0   lon     float64
 1   lat     float64
 2   evapr   float32
 3   lhtfl   float32
 4   nlwrs   float32
 5   hum2m   float32
 6   qnet    float32
 7   shtfl   float32
 8   nswrs   float32
 9   tmp2m   float32
 10  tmpsf   float32
 11  wnd10   float32
 12  year    int64  
 13  month   int64  
dtypes: float32(10), float64(2), int64(2)
memory usage: 2.0 GB


In [6]:
# sort targets for alignment with data
targets = labels.mhw_label \
    .stack(dim=["lon", "lat", "time"]) \
    .to_pandas().T.reset_index() 

targets = targets \
    .sort_values(by=["time"]) \
    .drop(["lon", "lat", "time"], axis=1)

In [7]:
# concatenate targets and drop rows that are completely empty of data.
df['targets'] = targets
all_nans = df.loc[:, variables[3:]].isna().all(1)
df = df.drop(df.index[all_nans], axis=0).fillna(-1)
targets = df.pop('targets')

In [8]:
X_train, X_test, y_train, y_test = \
    train_test_split(df, targets, test_size=0.3, shuffle=True, stratify=targets, random_state=1)

In [9]:
len(X_train), len(y_train)

(10613941, 10613941)

In [10]:
# set up model pipeline and cross validate.
scoring = ['accuracy', 'precision', 'recall', 'balanced_accuracy']

pipe = pipeline.Pipeline(
    [('pca', PCA(n_components=5)),
     ('scale', StandardScaler()),
     ('sample', RandomUnderSampler(random_state=1)),   # tried other strategies, but performed poorly or took too long. 
     ('clf', SGDClassifier(loss='log', verbose=10))],
    verbose=True
)

results = cross_validate(pipe, X_train, y_train, 
                         scoring=scoring, verbose=10, 
                         n_jobs=8, return_estimator=True)

print(results["test_recall"])

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed:  1.9min remaining:  2.8min
[Parallel(n_jobs=8)]: Done   3 out of   5 | elapsed:  1.9min remaining:  1.3min


[0.52606492 0.52752535 0.48452271 0.41635518 0.42840078]


[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed:  2.0min remaining:    0.0s
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed:  2.0min finished


In [12]:
best_model = results["estimator"][np.argmax(results["test_recall"])]
y_pred = best_model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.94      0.62      0.75   4173904
         1.0       0.11      0.53      0.18    374928

    accuracy                           0.62   4548832
   macro avg       0.52      0.58      0.47   4548832
weighted avg       0.87      0.62      0.70   4548832



Undersampling w/o bootstrap

```
              precision    recall  f1-score   support

         0.0       0.94      0.62      0.75   4173904
         1.0       0.11      0.53      0.18    374928

    accuracy                           0.61   4548832
   macro avg       0.52      0.57      0.47   4548832
weighted avg       0.87      0.61      0.70   4548832
```

w bootstrap
```
              precision    recall  f1-score   support

         0.0       0.94      0.62      0.74   4173904
         1.0       0.11      0.53      0.18    374928

    accuracy                           0.61   4548832
   macro avg       0.52      0.58      0.46   4548832
weighted avg       0.87      0.61      0.70   4548832
```